# Setting librerie

In [ ]:
from google.colab import drive

import pandas as pd 
import numpy as np 
from collections import Counter 
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import re

import nltk
from nltk.tokenize import word_tokenize 
nltk.download('punkt')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
from nltk.corpus import stopwords

from wordcloud import WordCloud 

from sklearn.utils import resample
from sklearn.decomposition import TruncatedSVD 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import v_measure_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora

!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle 
import os

# Caricamento Dataset

In [ ]:
drive.mount("/content/gdrive")

In [ ]:
food_reviews_df = pd.read_csv('/content/gdrive/MyDrive/Progetto_Text_Mining/Reviews.csv', sep = ",") 
food_reviews_df.head()

#Analisi esplorativa iniziale

In [ ]:
print(food_reviews_df.shape)

Il dataset è composto da 568454 record (recensioni) e 10 features

In [ ]:
print(len(food_reviews_df['UserId'].unique()))
print(len(food_reviews_df['ProductId'].unique()))

Il dataset è composto da 74258 prodotti univoci e 256059 utenti univoci

In [ ]:
food_reviews_df[['HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score']].describe()

Si osserva:


*   il range per HelpfulnessNumerator è [0, 866.000000]
*   il range per HelpfulnessDenominator è [0, 923.00000]
*   il range per HelpfScoreulnessNumerator è [0, 5] con valore medio pari a 4.183199




In [ ]:
print(min(pd.to_datetime(food_reviews_df.Time,  unit='s')))
print(max(pd.to_datetime(food_reviews_df.Time,  unit='s')))

La recensione più datata nel dataset risale al 1999-10-08 mentre la più recente al 2012-10-26

# Data Cleaning

## Rimozione dei duplicati





In [ ]:
food_reviews_df.duplicated(subset = ["UserId","ProfileName","Time","Text"]).sum()

Sono presenti 174521 record duplicati, si procede con l'eliminazione di essi.

In [ ]:
food_reviews_df_no_duplicates = food_reviews_df.drop_duplicates(subset = ["UserId","ProfileName","Time","Text"])

In [ ]:
100 - (food_reviews_df_no_duplicates['Id'].size*1.0)/(food_reviews_df['Id'].size*1.0)*100

Si va così ad eliminare circa il 30% delle osservazioni.

## Rimozione inconsistenze (HelpfulnessNumerator e HelpfulnessDenominator) 

Si notano delle inconsistenze nella feature Helpfullness, il denominatore infatti non può essere minore del numeratore. Si è proceduto ad eliminarli

In [ ]:
food_reviews_df_no_duplicates[food_reviews_df_no_duplicates["HelpfulnessNumerator"] > food_reviews_df_no_duplicates["HelpfulnessDenominator"]]

In [ ]:
food_reviews_df_no_duplicates = food_reviews_df_no_duplicates[food_reviews_df_no_duplicates["HelpfulnessNumerator"] <= food_reviews_df_no_duplicates["HelpfulnessDenominator"]]

## Gestione missing value

Si osserva la presenza di missing value considerando in particolare le feature Score, Summary, Text.

In [ ]:
food_reviews_df_no_duplicates[['Score', 'Summary', 'Text']].isnull().sum()

Si va più nello specifico ad analizzare tali record

In [ ]:
food_reviews_df_no_duplicates[food_reviews_df_no_duplicates['Summary'].isnull()]

Al posto di eliminare i record con testi vuoti si è preferito mettere una stringa vuota e considerarla. Viene poi verificata nuovamente la presenza di missing values.

In [ ]:
food_reviews_df_no_duplicates["Summary"] = food_reviews_df_no_duplicates["Summary"].fillna('')

In [ ]:
food_reviews_df_no_duplicates[['Score', 'Summary', 'Text']].isnull().sum()

# Analisi esplorativa

Dopo aver gestito record duplicati, incosistenze e missing values si riprone un'analisi esplorativa dei dati.

## Grafico distribuzione numero di recensioni in base al rating

Si analizza la distribuzione della variabile Score, osservando quante recensioni ci sono epr ciascuno dei valori che assume la variabile Score

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(food_reviews_df_no_duplicates['Score'])
plt.title("Distribuzione della variabile Score", fontweight='bold')
plt.xlabel("Score")
plt.ylabel("Numero di recensioni")

In [ ]:
# numero di recensioni per Score
print(food_reviews_df_no_duplicates['Score'].value_counts().sort_index());

Si osserva che la maggior parte delle recensioni hanno come valore di rating 5

## Violin Plot

Si analizza per ciascun valore assunto dalla variabile Score il numero di parole delle relative recensioni.

In [ ]:
food_reviews_df_no_duplicates["Review_Word_Count"] = food_reviews_df_no_duplicates["Text"].apply(lambda x: len(x.split()))

# ad esempio il testo della recensione considerata contiene 48 parole
with pd.option_context('display.max_colwidth', -1):
    display(food_reviews_df_no_duplicates.head(1))

In [ ]:
plt.figure(figsize=(20,8))
ax = sns.violinplot(y='Review_Word_Count', x='Score', data=food_reviews_df_no_duplicates)
ax.set_title('Distribuzione numero parole per Score', fontweight='bold')
ax.set_ylabel("Numero parole")
ax.set_xlabel("Score")

Si analizzano con più attenzione tutte quelle recensioni il cui testo ha un numero di parole inferiori a 500.

In [ ]:
plt.figure(figsize=(20,8))
df_temp = food_reviews_df_no_duplicates[food_reviews_df_no_duplicates['Review_Word_Count'] < 500]
ax2 = sns.violinplot(y='Review_Word_Count', x='Score', data=df_temp)
ax2.set_title('Distribuzione numero parole per Score', fontweight='bold')
ax2.set_ylabel("Numero parole")
ax2.set_xlabel("Score")

Si calcolano i valori mediani del numero di parole per recensioni per ogni valore di Score

In [ ]:
for i in range(1,6):
  print(food_reviews_df_no_duplicates[food_reviews_df_no_duplicates.Score==i].Review_Word_Count.median())

## Distribuzione recensioni nel tempo

Si analizza come sono distribuite le recensioni nel tempo.

In [ ]:
food_reviews_df_no_duplicates["Time"] = pd.to_datetime(food_reviews_df_no_duplicates.Time,  unit='s')
plt.figure(figsize=(10,5))
plt.hist(food_reviews_df_no_duplicates["Time"].dt.year, bins=50)
plt.ylabel('Numero recensioni')
plt.xlabel('Anno')
plt.title('Distribuzione del numero di recensioni negli anni', fontweight='bold', fontsize=10)

Si osserva che la maggiorparte delle recensioni risale agli ultimi anni disponibili nel dataset.

## Usefullness plot

Si procede nel creare una nuova feature, ovvero PercentHelpful. Essa indica per un prodotto quante delle recensioni prodotte sono state indicate come effettivamente utili.

In [ ]:
food_reviews_df_no_duplicates["PercentHelpful"] = food_reviews_df_no_duplicates["HelpfulnessNumerator"] / food_reviews_df_no_duplicates["HelpfulnessDenominator"]*100
df_usefullness_plot=food_reviews_df_no_duplicates[food_reviews_df_no_duplicates["PercentHelpful"].notnull()]

In [ ]:
plt.figure(figsize=(10,5))

plt.hist(df_usefullness_plot["PercentHelpful"], bins=101)
plt.ylabel('Numero dello recensioni')
plt.xlabel('Percentuale di utilità')
plt.title('Percentuale di utilità delle recensioni', fontweight='bold', fontsize=10)
plt.show()

Si osserva che nella maggiorparte delle recensioni si un valore di PercentHelpful pari a 100.

## Word cloud

Si genera un Word Cloud sulla totalità dei testi delle recensioni per notare se sono presenti ad alto livello dei pattern potenzialmente ricorrenti.

In [ ]:
text_values = food_reviews_df_no_duplicates['Text'].values 

wordcloud = WordCloud(width=800, height=400, collocations=False).generate(str(text_values))

plt.figure(figsize=(10,5))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

# Text pre processing

## Normalization

Si procede nella creazione di singole funzioni che saranno poi accorpate in una macro funzione.

### Conversione in caratteri minuscoli

In [ ]:
def to_lower(phrase):
  phrase = phrase.lower()
  return phrase

### Rimozione emoji

In [ ]:
def remove_emoji(phrase):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', phrase)

### Rimozione di numeri

In [ ]:
def rimozione_numbers(phrase):
  removed_numbers=re.sub(r'\d+', '',phrase)
  return phrase

### Espansione delle contrazioni

In [ ]:
def expand_eng_contradictions(phrase):
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

### Rimozione HTML

In [ ]:
def remove_html_tags(sentence):
    pattern = re.compile("<.*?>") 
    cleaned_sentence = re.sub(pattern,'',sentence).strip()
    return cleaned_sentence

def remove_html_entities(sentence):
    pattern = re.compile("&[a-z0-9]+|&#[0-9]{1,6}|&#x[0-9a-f]{1,6}")
    cleaned_sentence = re.sub(pattern,'',sentence).strip()
    return cleaned_sentence

### Rimozione URL 

In [ ]:
def remove_urls(sentence):
    http_pattern = re.compile(r"http\S+") 
    cleaned_sentence = re.sub(http_pattern,'',sentence).strip()
    www_pattern = re.compile(r"www\S+") 
    cleaned_sentence = re.sub(www_pattern,'',cleaned_sentence)
    return cleaned_sentence

### Rimozione di parole che presentano al loro interno 3 o più caratteri ripetuti

In [ ]:
def remove_words_with_repeated_characters(sentence): 
    pattern = re.compile("\\s*\\b(?=\\w*(\\w)\\1{2,})\\w*\\b")
    cleaned_text  = re.sub(pattern,' ',sentence)
    return (cleaned_text)

### Rimozione di punteggiatura e caratteri speciali

In [ ]:
def remove_special_characters_punctuations(sentence):
    pattern = re.compile("[^a-zA-Z]+") 
    cleaned_text  = re.sub(pattern,' ',sentence).strip()
    return cleaned_text

### Macro Funzione Normalization

In [ ]:
def funzione_finale_normalizzazione(text):
  text = to_lower(text)
  text = remove_emoji(text)
  text = rimozione_numbers(text)
  text = expand_eng_contradictions(text)
  text = remove_html_tags(text)
  text = remove_html_entities(text)
  text = remove_urls(text)
  text = remove_words_with_repeated_characters(text)
  text = remove_special_characters_punctuations(text)
  text = text.lower()
  return text

In [ ]:
df_normalized = food_reviews_df_no_duplicates
df_normalized['Text_normalized'] = food_reviews_df_no_duplicates['Text'].apply(funzione_finale_normalizzazione)

## Tokenization

In [ ]:
# NLTK tokenizer
df_tokenized = df_normalized
df_tokenized['Token_Text'] = df_normalized['Text_normalized'].apply(word_tokenize) 

## Stop-words removal

In [ ]:
def rimozione_stop_words(text, stopwords_list):
  return ([word for word in text if word not in stopwords_list])

In [ ]:
nltk.download('stopwords')

stopwords_list = stopwords.words('english')

df_no_stop_words = df_tokenized
df_no_stop_words['Text_no_stop_words'] = df_tokenized['Token_Text'].apply(lambda x: rimozione_stop_words(x, stopwords_list))

## Lemmatization

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

def lemmaSentence(token_words):
    lemma_text=[]
    for word in token_words:
        lemma_text.append(wordnet_lemmatizer.lemmatize(word))
    return lemma_text
  

df_lemmatized = df_no_stop_words
df_lemmatized['Text_lemmatized'] = df_no_stop_words['Text_no_stop_words'].apply(lemmaSentence)

In [ ]:
del df_tokenized, df_normalized, df_no_stop_words

# Classificazione

## Preparazione dati per classificazione

### Conversione in binario della feature Score

Si procede nel trasformare la feature Score (che ha valori 1:5) in una feature binaria (0,1)

In [ ]:
df_classificazione = df_lemmatized[["Score", "Text_lemmatized"]]
df_classificazione.head()

In [ ]:
df_classificazione = df_classificazione[df_classificazione["Score"]!=3]
df_classificazione.head()

In [ ]:
df_classificazione.loc[df_classificazione.Score < 3, "Score"] = 0
df_classificazione.loc[df_classificazione.Score > 3, "Score"] = 1
df_classificazione.head()

### Divisione train e test set

Si procede con al divisione del dataset in train e test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_classificazione["Text_lemmatized"], 
                                                    df_classificazione["Score"], 
                                                    test_size=0.33, 
                                                    random_state=1, 
                                                    stratify=df_classificazione['Score'])

### Gestione della class imbalance

Prendendo in considerazione il training set si nota un problema di class imbalance

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y_train)
plt.title("Distribuzione Score binario", fontweight='bold', fontsize=15)
plt.xlabel("Score")
plt.ylabel("Numero delle recensioni")

In [ ]:
df_class_imbalance = pd.concat([X_train, y_train], axis = 1)
df_class_imbalance.head()

In [ ]:
df_majority = df_class_imbalance[df_class_imbalance.Score==1]
df_minority = df_class_imbalance[df_class_imbalance.Score==0]

df_majority_downsampled = resample(df_majority, replace=False, 
                                   n_samples=df_minority.shape[0], random_state=123) 

Si verifica la cardinalità delle due classi dopo aver eseguito downsampling

In [ ]:
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
df_downsampled.Score.value_counts()

In [ ]:
X_train_downsampled = df_downsampled["Text_lemmatized"]
y_train_downsampled = df_downsampled["Score"]

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(df_downsampled["Score"])
plt.title("Distribuzione Score binario", fontweight='bold', fontsize=15)
plt.xlabel("Score")
plt.ylabel("Numero delle recensioni")

### Text representation (train)

#### BOW

In [ ]:
cv = CountVectorizer(min_df=0., max_df=1.0) 
cv_model = cv.fit(X_train_downsampled.astype(str)) 
X_train_bow = cv_model.transform(X_train_downsampled.astype(str))

In [ ]:
X_train_bow.shape

In [ ]:
print(X_train_bow.min())
print(X_train_bow.max())

#### TF-IDF

In [ ]:
tv = TfidfVectorizer(min_df=5, max_df=1., use_idf=True)
tv_model = tv.fit(X_train_downsampled.astype(str))
X_train_tv = tv.transform(X_train_downsampled.astype(str))

In [ ]:
X_train_tv.shape

In [ ]:
print(X_train_tv.min())
print(X_train_tv.max())

### Dimensionality reduction

#### BOW

In [ ]:
truncatedSVD_bow = TruncatedSVD(n_components = 1000)
svd_bow = truncatedSVD_bow.fit(X_train_bow)
X_train_bow = truncatedSVD_bow.transform(X_train_bow)
print(svd_bow.explained_variance_ratio_.sum()) # explained variance

#### TF-IDF

In [ ]:
truncatedSVD_tfidf = TruncatedSVD(n_components = 2000)
svd_tfidf = truncatedSVD_tfidf.fit(X_train_tv)
X_train_tv = truncatedSVD_tfidf.transform(X_train_tv)
print(svd_tfidf.explained_variance_ratio_.sum()) # explained variance

### Applicazione delle trasformazioni al test set

#### BOW

In [ ]:
X_test_bow = cv_model.transform(X_test.astype(str))
X_test_bow = svd_bow.transform(X_test_bow)

#### TF-IDF

In [ ]:
X_test_tv = tv.transform(X_test.astype(str))
X_test_tv = truncatedSVD_tfidf.transform(X_test_tv)

## Implementazione algoritmi di classificazione

In [ ]:
def result_evaluation(prediction, target_value):
  print(confusion_matrix(target_value, prediction))
  print(classification_report(target_value, prediction))

### Logistic Regression

#### BOW

In [ ]:
lr_bow = LogisticRegression(random_state=123)
lr_bow_model = lr_bow.fit(X_train_bow, y_train_downsampled)
lr_pred_bow = lr_bow_model.predict(X_test_bow)

In [ ]:
result_evaluation(lr_pred_bow, y_test)

In [ ]:
from sklearn.metrics import precision_score
print('Precision: %.3f' % precision_score(y_test, lr_pred_bow))

#### TF-IDF

In [ ]:
lr_tfidf = LogisticRegression(random_state=123)
lr_tfidf_model = lr_tfidf.fit(X_train_tv, y_train_downsampled)
lr_pred_tfidf = lr_tfidf_model.predict(X_test_tv)

In [ ]:
result_evaluation(lr_pred_tfidf, y_test)

### SVM

Si procede utilizzando una versione approssimata: SGDClassifier 
(https://scikit-learn.org/stable/modules/kernel_approximation.html)



#### BOW

In [ ]:
rbf_feature_bow = RBFSampler(gamma=1, random_state=1)
X_features_bow = rbf_feature_bow.fit_transform(X_train_bow)
svm_bow = SGDClassifier(max_iter=25)

svm_bow_model = svm_bow.fit(X_features_bow, y_train_downsampled)
X_features_bow_test = rbf_feature_bow.transform(X_test_bow)
svm_pred_bow = svm_bow_model.predict(X_features_bow_test)

In [ ]:
result_evaluation(svm_pred_bow, y_test)

#### TF-IDF

In [ ]:
rbf_feature_tfidf = RBFSampler(gamma=1, random_state=1)
X_features_tfidf = rbf_feature_tfidf.fit_transform(X_train_tv)
svm_tfidf = SGDClassifier(max_iter=25)

svm_tfidf_model = svm_tfidf.fit(X_features_tfidf, y_train_downsampled)
X_features_tdfidf_test = rbf_feature_tfidf.transform(X_test_tv)
svm_pred_tfidf = svm_tfidf_model.predict(X_features_tdfidf_test)

In [ ]:
result_evaluation(svm_pred_tfidf, y_test)

### Random Forest

#### BOW

In [ ]:
rf_bow = RandomForestClassifier(n_jobs = -1)
rf_bow_model = rf_bow.fit(X_train_bow, y_train_downsampled)
rf_pred_bow = rf_bow_model.predict(X_test_bow)

In [ ]:
result_evaluation(rf_pred_bow, y_test)

#### TF-IDF

In [ ]:
rf_tfidf = RandomForestClassifier(n_jobs = -1)
rf_tfidf_model = rf_tfidf.fit(X_train_tv, y_train_downsampled)
rf_pred_tfidf = rf_tfidf_model.predict(X_test_tv)

In [ ]:
result_evaluation(rf_pred_tfidf, y_test)

# Clustering

### Preparazione dati per clustering

Dal momento in cui ci servono tutte le label per Score è necessario considerare i dati originali e non quelli usati per la classificazione binaria

In [ ]:
X_train_clust, X_test_clust, y_train_clust, y_test_clust = train_test_split(df_lemmatized['Text_lemmatized'], 
                                                    df_lemmatized['Score'], 
                                                    test_size=0.33,                                             
                                                    stratify=df_lemmatized['Score'],
                                                    random_state=123)

df_clust_train = pd.concat([X_train_clust, y_train_clust], axis = 1)

Si osserva uno sbilanciamento dei dati

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y_train_clust)
plt.title("Distribuzione della variabile Score", fontweight='bold')
plt.xlabel("Score")
plt.ylabel("Numero di recensioni")

Si procede con il bilanciamento (e shuffling) dei dati

In [ ]:
dim_score_2 = df_clust_train['Score'].value_counts()[2]

score_1 = df_clust_train.loc[df_clust_train['Score'] == 1].sample(dim_score_2)
score_2 = df_clust_train.loc[df_clust_train['Score'] == 2]
score_3 = df_clust_train.loc[df_clust_train['Score'] == 3].sample(dim_score_2)
score_4 = df_clust_train.loc[df_clust_train['Score'] == 4].sample(dim_score_2)
score_5 = df_clust_train.loc[df_clust_train['Score'] == 5].sample(dim_score_2) 

df_clust_train_balanced = pd.concat([score_1, score_2, score_3, score_4, score_5])

In [ ]:
# shuffling 
df_clust_train_balanced = shuffle(df_clust_train_balanced, random_state = 123)

X_train_clust_balanced = df_clust_train_balanced['Text_lemmatized']
y_train_clust_balanced = df_clust_train_balanced['Score']

Si ottengono così i dati bilanciati

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x = y_train_clust_balanced)
plt.title("Distribuzione della variabile Score", fontweight='bold')
plt.xlabel("Score")
plt.ylabel("Count")

Si procede con la rappresentazione tf-idf dei dati bilaniciati

In [ ]:
tv_clust = TfidfVectorizer(min_df = 6)
tv_model_clust = tv_clust.fit(X_train_clust_balanced.astype(str))

X_train_clust_tv = tv_model_clust.transform(X_train_clust_balanced.astype(str))

Si procede successivamente con la riduzione della dimensionalità, considerati i limiti computazionali per l'esecuzioned degli algoritmi di clustering considerati

In [ ]:
svd_tfidf_clust = TruncatedSVD(n_components=1500)
svd_tfidf_clust = svd_tfidf_clust.fit(X_train_clust_tv)

svd_tfidf_clust.explained_variance_ratio_.sum()

X_train_clust_tv = svd_tfidf_clust.transform(X_train_clust_tv)

Si procede con la normalizzazione dei dati

In [ ]:
X_train_clust_stand_tv = preprocessing.normalize(X_train_clust_tv)

### K-means

Si applica l'algoritmo

In [ ]:
num_cluster = 5
km = KMeans(n_clusters=num_cluster, max_iter=20, n_init=10, random_state=123)
km = km.fit(X_train_clust_stand_tv)
y_clus_km = km.predict(X_train_clust_stand_tv)

In [ ]:
print(f'''V measure- {v_measure_score(y_clus_km, y_train_clust_balanced)}''')
print(f'''ARI- {adjusted_rand_score(y_clus_km,y_train_clust_balanced)}''')

In [ ]:
num_recensioni_labels_km = Counter(km.labels_)

print('Numero recensioni nei cluster')

for i in range(0, num_cluster):
  print(str(i) + ' ' + str(num_recensioni_labels_km[i]))

In [ ]:
# per visualizzazione
for i in range(num_cluster):
  text = " ".join(str(t) for t in np.where(km.labels_ == i, X_train_clust_balanced, ''))
  text = text.replace("'", "")

  wordcloud = WordCloud(width=800, height=400, collocations=False).generate(str(text))

  plt.imshow(wordcloud)
  plt.axis("off")
  plt.show()

### Agglomerative clustering (una tiplogia di clustering gerarchico)

In [ ]:
num_cluster = 5

ag = AgglomerativeClustering(n_clusters = num_cluster, affinity='euclidean', linkage='ward')
ag.fit(X_train_clust_stand_tv[:20000])

In [ ]:
y_clus_ag = ag.labels_
print(f'''V measure- {v_measure_score(y_clus_ag, y_train_clust_balanced[:20000])}''')
print(f'''ARI- {adjusted_rand_score(y_clus_ag,y_train_clust_balanced[:20000])}''')

In [ ]:
num_recensioni_labels_ag = Counter(ag.labels_)

print('Numero recensioni nei cluster')

for i in range(0, num_cluster):
  print(str(i) + ' ' + str(num_recensioni_labels_ag[i]))

In [ ]:
# per visualizzazione
for i in range(num_cluster):
  text = " ".join(str(t) for t in np.where(ag.labels_ == i, X_train_clust_balanced[:20000], ''))
  text = text.replace("'", "")

  wordcloud = WordCloud(width=800, height=400, collocations=False).generate(str(text))
  
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.show()

# Single-Label Multi-Class Classification

### Random Forest

Si procede con la rappresentazione (tf-idf) e riduzione della dimensionalità del test set


In [ ]:
X_train_multiclass_tv = X_train_clust_tv
y_train_multiclass = y_train_clust_balanced
X_test_multiclass = X_test_clust
y_test_multiclass = y_test_clust

In [ ]:
X_test_multiclass_tv = tv_clust.transform(X_test_multiclass.astype(str))

X_test_multiclass_tv = svd_tfidf_clust.transform(X_test_multiclass_tv)

Si applica l'algoritmo Random Forest

In [ ]:
rf_multi_class_tfidf = RandomForestClassifier(n_jobs = -1)
rf_multi_class_model = rf_multi_class_tfidf.fit(X_train_multiclass_tv, y_train_multiclass)

rf_multi_class_pred_tfidf = rf_multi_class_model.predict(X_test_multiclass_tv)

In [ ]:
result_evaluation(rf_multi_class_pred_tfidf, y_test_multiclass)

# Topic Modelling

### LDA

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data = df_lemmatized['Text_lemmatized'].values.tolist()

data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

In [ ]:
# number of topics
num_topics = 5

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus = corpus,
                                       id2word = id2word,
                                       num_topics = num_topics)
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('/content/ldavis_prepared_'+str(num_topics))

if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, '/content/ldavis_prepared_'+ str(num_topics) +'.html')

LDAvis_prepared